<a href="https://colab.research.google.com/github/NicholasLeotta99/Neuromatch_BrainCancer/blob/Nick/code/VGG19_Classifier_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Package Imports
import torch
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.utils import make_grid
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, TensorDataset
import torchvision.models as models
from torchsummary import summary

import os
from google.colab import files
from PIL import Image
import subprocess

In [2]:
# @title Data Loading
print('Upload kaggle.json account information to download the dataset:')
files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset
!unzip -q brain-tumor-mri-dataset.zip

Upload kaggle.json account information to download the dataset:


Saving kaggle.json to kaggle.json
 99% 147M/149M [00:09<00:00, 21.2MB/s]
100% 149M/149M [00:09<00:00, 16.4MB/s]


In [3]:
'''
# @title Download and processing Zenodo data - unneeded, data represented in original dataset
#!wget https://zenodo.org/record/7047164/files/adnoma.zip
#!wget https://zenodo.org/record/7047164/files/Gliomas.zip
#!wget https://zenodo.org/record/7047164/files/Meningioma.zip

def grab_format_zenodo(type='adnoma', size=(256,256)):
  url = f'https://zenodo.org/record/7047164/files/{type}.zip'
  print(f'Downloading {type}')
  subprocess.call(['wget', '-v', url])
  print(f'{type} acquired, unzipping...')
  subprocess.call(['unzip', '-q', f'{type}.zip'])
  files = os.listdir(f'./{type}')
  print(f'Unzipping {type} completed [{len(files)} samples extracted], beginning conversion...')
  size=(256,256)
  for i in files:
    im = Image.open(f'./{type}/{i}')
    i, _ = i.split('.')
    new = im.resize(size)
    if not os.path.isdir(f'./{type}_png'):
      os.mkdir(f'./{type}_png')
    new.save(f'./{type}_png/{i}.png', 'png')

# for i in ['adnoma', 'Gliomas', 'Meningioma']:
#   grab_format_zenodo(type=i)
'''

"\n# @title Download and processing Zenodo data - unneeded, data represented in original dataset\n#!wget https://zenodo.org/record/7047164/files/adnoma.zip\n#!wget https://zenodo.org/record/7047164/files/Gliomas.zip\n#!wget https://zenodo.org/record/7047164/files/Meningioma.zip\n\ndef grab_format_zenodo(type='adnoma', size=(256,256)):\n  url = f'https://zenodo.org/record/7047164/files/{type}.zip'\n  print(f'Downloading {type}')\n  subprocess.call(['wget', '-v', url])\n  print(f'{type} acquired, unzipping...')\n  subprocess.call(['unzip', '-q', f'{type}.zip'])\n  files = os.listdir(f'./{type}')\n  print(f'Unzipping {type} completed [{len(files)} samples extracted], beginning conversion...')\n  size=(256,256)\n  for i in files:\n    im = Image.open(f'./{type}/{i}')\n    i, _ = i.split('.')\n    new = im.resize(size)\n    if not os.path.isdir(f'./{type}_png'):\n      os.mkdir(f'./{type}_png')\n    new.save(f'./{type}_png/{i}.png', 'png')\n\n# for i in ['adnoma', 'Gliomas', 'Meningioma']:\

In [4]:
# @title Loading Model
Classifier = models.vgg19(weights='DEFAULT')
#Replace the classifying linear network with one with custom number of output classes
num_features = Classifier.classifier[-1].in_features
Classifier.classifier[-1] = nn.Linear(num_features, 4)
Classifier.to('cuda')
summary(Classifier, (3, 256, 256))

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:08<00:00, 70.9MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
              ReLU-2         [-1, 64, 256, 256]               0
            Conv2d-3         [-1, 64, 256, 256]          36,928
              ReLU-4         [-1, 64, 256, 256]               0
         MaxPool2d-5         [-1, 64, 128, 128]               0
            Conv2d-6        [-1, 128, 128, 128]          73,856
              ReLU-7        [-1, 128, 128, 128]               0
            Conv2d-8        [-1, 128, 128, 128]         147,584
              ReLU-9        [-1, 128, 128, 128]               0
        MaxPool2d-10          [-1, 128, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]         295,168
             ReLU-12          [-1, 256, 64, 64]               0
           Conv2d-13          [-1, 256, 64, 64]         590,080
             ReLU-14          [-1, 256,

In [5]:
classes = os.listdir('./Training/')

0 tensor([3, 1, 0, 2, 1, 1, 0, 2, 1, 1])


In [6]:
# @title Dataset Generation
#TODO - Verify labels correspond to classes
data_transf = transforms.Compose([
transforms.Resize((256, 256)),
transforms.ToTensor()])
import matplotlib.pyplot as plt
TrainingDataset = ImageFolder(root='./Training/', transform=data_transf)#, transform=torchvision.models.VGG19_Weights.IMAGENET1K_V1.transforms)
TrainingLoader = DataLoader(TrainingDataset, batch_size=10, shuffle=True)
for batch_number , (images, labels) in enumerate(TrainingLoader):
  print(batch_number, labels)
  plt.imshow(torch.moveaxis(images[0], 0, -1))
  plt.title(classes[labels[0]])
  plt.show()
  break

In [11]:
# Cross Entropy Loss for categorization
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(Classifier.parameters(), lr=0.005, momentum=.9)

def one_epoch(epoch_index):
  current_loss = 0

  for i, data in enumerate(TrainingLoader):
    inputs, labels = data
    inputs = inputs.to('cuda')
    labels = labels.to('cuda')

    optimizer.zero_grad()
    outputs = Classifier(inputs)

    loss = loss_fn(outputs, labels)
    loss.backward()

    optimizer.step()

    current_loss += loss.item()
  return current_loss

In [12]:
for i in range(0,100):
  loss = one_epoch(i)
  print(loss)


138.063372494129
85.54820427135564
81.4352500745299
51.437024059563555
29.682250680642028
47.22872790036581
32.415896601477925
21.398940484864056
3.881060488690899
27.779968600611866
8.285132172040392


KeyboardInterrupt: ignored